In [13]:
#import necessary libraries
import pandas as pd
import requests
from bs4 import BeautifulSoup
!pip install altair
import altair as alt

In [14]:
#create function for scraping website
url = 'https://github.com/telegeography/www.submarinecablemap.com/tree/master/public/api/v2/cable'

def Parse_to_DataFrame(url):
    
    parse_html = BeautifulSoup(requests.get(url).text)
    
    link_list = []
    
    #retrieve links for each cables page 
    for link in parse_html.find_all('a', attrs={'class':'js-navigation-open link-gray-dark'}):
        
        link_list.append(link.get('href'))
        
    html_list = []
    
    #retrieve line of html with cable json data
    for i in link_list:
    
        print('processing json from', i)
    
        example_html = BeautifulSoup(requests.get('https://github.com'+ i).text)

        html2 = example_html.find('td', attrs={'class':'blob-code blob-code-inner js-file-line'})
    
        html_list.append(html2)
    
    
    html_list.remove(html_list[77]) #remove html containing map, not cable json data
    html_list.remove(html_list[12]) #remove html containing json data with list of cable names
    
    cable_df_list = [] 
    
    count = 0    

    #convert json files to one dataframe
    while count < len(html_list):
    
        text = html_list[count].get_text()
    
        df = pd.read_json(text)
        
        cable_df_list.append(df)

        count+= 1    
        
    cable_df = pd.concat(cable_df_list, ignore_index=True)
    
    #create Country column 
    for i in cable_df.index:
    
        dict_key = cable_df.at[i, 'landing_points']
        Country = dict_key['name'].split(',')
        cable_df.at[i, 'Country'] = Country[-1]
        
    return cable_df

final_df = Parse_to_DataFrame(url)

processing json from /telegeography/www.submarinecablemap.com/blob/master/public/api/v2/cable/2africa.json
processing json from /telegeography/www.submarinecablemap.com/blob/master/public/api/v2/cable/acs-alaska-oregon-network-akorn.json
processing json from /telegeography/www.submarinecablemap.com/blob/master/public/api/v2/cable/aden-djibouti.json
processing json from /telegeography/www.submarinecablemap.com/blob/master/public/api/v2/cable/adria-1.json
processing json from /telegeography/www.submarinecablemap.com/blob/master/public/api/v2/cable/aeconnect-1.json
processing json from /telegeography/www.submarinecablemap.com/blob/master/public/api/v2/cable/africa-coast-to-europe-ace.json
processing json from /telegeography/www.submarinecablemap.com/blob/master/public/api/v2/cable/alaska-united-east-au-east.json
processing json from /telegeography/www.submarinecablemap.com/blob/master/public/api/v2/cable/alaska-united-southeast-au-se.json
processing json from /telegeography/www.submarinec

In [64]:
#create dataframe with number of cables for each country
country_cable_df = pd.DataFrame(final_df['Country'].value_counts()).reset_index()
country_cable_df = country_cable_df.rename(columns = {'index':'Country', 'Country':'# Cables'})
country_cable_df

,Country,# Cables
0,United States,194
1,Indonesia,189
2,United Kingdom,115
3,Philippines,61
4,Spain,54
...,...,...
174,Sierra Leone,1
175,Bulgaria,1
176,Liberia,1
177,Mauritania,1


In [36]:
#read in csv file with GDP growth data
gdp_q2 = pd.read_csv('economic-decline-in-the-second-quarter-of-2020.csv')
gdp_q2 = gdp_q2.rename(columns = {'Entity':'Country'})
gdp_q2

,Country,Code,"GDP growth from previous year, 2020 Q2"
0,Austria,AUT,-13.300000
1,Belgium,BEL,-14.500000
2,Bulgaria,BGR,-8.200000
3,Canada,CAN,-13.494697
4,Chile,CHL,-13.682773
5,China,CHN,3.200000
6,Colombia,COL,-15.700000
7,Cyprus,CYP,-11.900000
8,Czech Republic,CZE,-10.700000
9,Denmark,DNK,-8.500000


In [142]:
#merge GDP file with cable dataframe
df_merge = country_cable_df.merge(gdp_q2, how = 'outer', on='Country')
df_merge.sample(20)

,Country,# Cables,Code,"GDP growth from previous year, 2020 Q2"
152,Saint Lucia,2.0,NaN,NaN
62,Trinidad and Tobago,7.0,NaN,NaN
1,Indonesia,189.0,IDN,-5.367830
60,Algeria,8.0,NaN,NaN
178,Ukraine,1.0,NaN,NaN
139,Benin,2.0,NaN,NaN
20,Chile,22.0,CHL,-13.682773
57,Equatorial Guinea,8.0,NaN,NaN
29,Finland,18.0,FIN,-5.200000
77,Guernsey,6.0,NaN,NaN


In [140]:
#scrape website for coastline data
url2 = 'http://chartsbin.com/view/ofv'

parse_html2 = BeautifulSoup(requests.get(url2).text)

table_html = parse_html2.find_all('tr')

#removes non-table information
table_html.remove(table_html[-1])
table_html.remove(table_html[-1])


In [243]:
#convert html to dictionary and then dataframe
empty_dict = {'Country':[],'Coastline (km)':[]}

#create list of column data
country_list = []

coast_list = []

for i in table_html:
     
    country_list.append(i.find('td', attrs={'class':'cname'}).text) 
    coast_list.append(i.find('td', attrs={'class':'tvalue tnumber tunit'}).text)
    
#append lists to dictionary
empty_dict['Country'] = country_list
empty_dict['Coastline (km)'] = coast_list

coastline = pd.DataFrame(empty_dict)
coastline = coastline.drop([0])

#change South Korea name in coastline database
for i in coastline.index:
    if coastline.at[i, 'Country'] == 'Korea, South':
        coastline.at[i, 'Country'] = 'South Korea'

coastline


,Country,Coastline (km)
1,Monaco,4 km
2,Gibraltar,12 km
3,Bosnia and Herzegovina,20 km
4,Tuvalu,24 km
5,Cocos (Keeling) Islands,26 km
...,...,...
189,Philippines,"36,289 km"
190,Russia,"37,653 km"
191,Greenland,"44,087 km"
192,Indonesia,"54,716 km"


In [174]:
#merge coastline dataframe with cable and gdp dataframe
df_merge2 = df_merge.merge(coastline, how = 'outer', on='Country')
df_merge2

,Country,# Cables,Code,"GDP growth from previous year, 2020 Q2",Coastline (km)
0,United States,194.0,USA,-9.50000,"19,924 km"
1,Indonesia,189.0,IDN,-5.36783,"54,716 km"
2,United Kingdom,115.0,GBR,-21.70000,"12,429 km"
3,Philippines,61.0,PHL,-16.50000,"36,289 km"
4,Spain,54.0,ESP,-22.10000,"4,964 km"
...,...,...,...,...,...
217,"Korea, North",NaN,NaN,NaN,"2,495 km"
218,"Bahamas, The",NaN,NaN,NaN,"3,542 km"
219,Svalbard,NaN,NaN,NaN,"3,587 km"
220,"Micronesia, Federated States of",NaN,NaN,NaN,"6,112 km"


In [246]:
#remove non-country rows and change NaN's to 0 
final_table = df_merge2[pd.notnull(df_merge2['GDP growth from previous year, 2020 Q2'])]
drop_list = ['OECD - Total', 'NAFTA', 'G7', 'Euro area', 'EU' ]

final_table = final_table[(final_table['Country'] != 'OECD - Total') & (final_table['Country'] != 'NAFTA') & (final_table['Country'] != 'G7') & (final_table['Country'] != 'Euro area') & (final_table['Country'] != 'EU') ]
final_table['Coastline (km)'] = final_table['Coastline (km)'].fillna(0).astype(str)

#convert coastline column to int and create cable per 1000km of coastline column
for i in final_table.index:
    
    final_table.at[i, 'Coastline (km)'] = final_table.at[i, 'Coastline (km)'].replace('km','').replace(',','')

final_table['Coastline (km)'] = final_table['Coastline (km)'].astype(int)
final_table['Cable per 1000km of coastline'] = (final_table['# Cables']/final_table['Coastline (km)'])*1000
final_table['Cable per 1000km of coastline'] = final_table['Cable per 1000km of coastline'].fillna(0).astype(float)
final_table

,Country,# Cables,Code,"GDP growth from previous year, 2020 Q2",Coastline (km),Cable per 1000km of coastline
0,United States,194.0,USA,-9.500000,19924,9.737001
1,Indonesia,189.0,IDN,-5.367830,54716,3.454200
2,United Kingdom,115.0,GBR,-21.700000,12429,9.252555
3,Philippines,61.0,PHL,-16.500000,36289,1.680950
4,Spain,54.0,ESP,-22.100000,4964,10.878324
5,Japan,50.0,JPN,-10.001380,29751,1.680616
6,Norway,42.0,NOR,-5.331291,25148,1.670113
8,China,40.0,CHN,3.200000,14500,2.758621
9,Denmark,40.0,DNK,-8.500000,7314,5.468964
10,Italy,32.0,ITA,-17.300000,7600,4.210526


In [247]:
#histogram of number of cables in each country
alt.Chart(final_table).mark_bar().encode(
    alt.X('Country', sort=alt.EncodingSortField(order='ascending')),
    alt.Y('# Cables'),
    tooltip = ['# Cables']
).interactive()


alt.Chart(...)

In [237]:
#create scatter graph and best fit line graph
final_table = final_table[final_table['Cable per 1000km of coastline'] != 0] #remove countries with no cables

base = alt.Chart(final_table).mark_circle().encode(
    x='Cable per 1000km of coastline',
    y='GDP growth from previous year, 2020 Q2',
    #size = alt.Size('Coastline (km)'),
    tooltip=['Country','# Cables', 'GDP growth from previous year, 2020 Q2', 'Cable per 1000km of coastline', 'Coastline (km)']
).interactive().properties(width=700, height=700)

line = base.transform_regression('Cable per 1000km of coastline', 'GDP growth from previous year, 2020 Q2',method="linear").mark_line(color="red")

base | line

alt.HConcatChart(...)